In [3]:
raw_data['縣市_鄉鎮市區'] = raw_data['縣市'] + '_' + raw_data['鄉鎮市區']

selected_X = train[feat_cols]
cat_X = raw_data[cat_cols+['單價']]
X = pd.concat([selected_X, cat_X], axis=1)
Y = pd.read_csv("../data/train_output.csv")
Y = np.log(Y)

test = pd.read_csv("../data/test_feat.csv")
private_org = pd.read_csv("../data/private_dataset_org.csv") 
private_test = pd.read_csv("../data/private_dataset.csv")
private_org['縣市_鄉鎮市區'] = private_org['縣市'] + '_' + private_org['鄉鎮市區']

raw_test_data = pd.read_csv("../data/public_dataset.csv")
raw_test_data['縣市_鄉鎮市區'] = raw_test_data['縣市'] + '_' + raw_test_data['鄉鎮市區']
selected_X = test[feat_cols]
selected_private_X = private_test[feat_cols]
cat_X = raw_test_data[cat_cols]
cat_X_private = private_org[cat_cols]
X_test = pd.concat([selected_X, cat_X], axis=1)
private_X_test = pd.concat([selected_private_X, cat_X_private], axis=1)

In [4]:
N_min = 10

for col in cat_cols:
    # print(f"now at {col}")
    le = LabelEncoder()
    tmp = np.concatenate([X[col], X_test[col]])
    le.fit(np.concatenate([tmp, private_X_test[col]]))
    X[col] = le.transform(X[col])
    X_test[col] = le.transform(X_test[col])
    private_X_test[col] = le.transform(private_X_test[col])

x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=507)
x_train.reset_index(inplace=True, drop=True)
x_valid.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_valid.reset_index(inplace=True, drop=True)
y_valid = np.exp(y_valid)


for c in cat_cols:
    # fit encoder
    be = BetaEncoder(c)
    be.fit(x_train, '單價')
    # mean
    feature_name = f'{c}_mean'
    x_train[feature_name] = be.transform(x_train, 'mean', N_min)
    x_valid[feature_name] = be.transform(x_valid, 'mean', N_min)
    X_test[feature_name] = be.transform(X_test, 'mean', N_min)
    private_X_test[feature_name] = be.transform(private_X_test, 'mean', N_min)

x_train = x_train.drop(['單價']+cat_cols,axis=1)
x_valid = x_valid.drop(['單價']+cat_cols,axis=1)
X_test = X_test.drop(cat_cols,axis=1)
private_X_test = private_X_test.drop(cat_cols,axis=1)

In [ ]:
print(
    f"Train shape: {x_train.shape}, {y_train.shape}, \
        Valid shape: {x_valid.shape}, {y_valid.shape}"
)

stack_model = stacking()
stack_model.fit(x_train, y_train)
y_pred = stack_model.predict(x_valid)
y_pred = np.exp(y_pred)
mape = mean_absolute_percentage_error(y_valid, y_pred)
print(f"MAPE: {mape * 100}")
public_pred = np.exp(stack_model.predict(X_test))
private_pred = np.exp(stack_model.predict(private_X_test))
sub_data.loc[:5876, 'predicted_price'] = public_pred
sub_data.loc[5876:, 'predicted_price'] = private_pred
sub_data.to_csv(f"{os.getcwd()}/data/public_private_submission_stack_v2.csv", index=False)

In [ ]:
sub_data['predicted_price'][:5876] = public_pred
sub_data['predicted_price'][5876:] = private_pred
sub_data.to_csv(f"../data/public_private_submission_stack_v2.csv", index=False)